In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [32]:
# TRIAL AND ERROR CODE

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def fetch_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve page: {url}")
        return None

def get_text_or_default(element, default="N/A"):
    return element.text.strip() if element else default

def fetch_detailed_page_details(detail_url):
    detailed_html_content = fetch_page(detail_url)
    if detailed_html_content:
        detail_soup = BeautifulSoup(detailed_html_content, 'html.parser')
        # list to get all strong tags
        strong_list = detail_soup.find_all('strong',class_='_3gHeV')
        #print(strong_list)
        Owner_type = get_text_or_default(strong_list[8]) if len(strong_list) > 8 else "N/A"
        Registration_ID = get_text_or_default(strong_list[2]) if len(strong_list) > 2 else "N/A"
        Car_Transmission = get_text_or_default(strong_list[6]) if len(strong_list) > 6 else "N/A"
        KM_Driven = get_text_or_default(strong_list[7]) if len(strong_list) > 7 else "N/A"
        Fuel_type = get_text_or_default(strong_list[9]) if len(strong_list) > 9 else "N/A"
        Downpayment_amount = get_text_or_default(detail_soup.find('label',class_='F6S7B'))
        Location = get_text_or_default(detail_soup.find('li',class_='_1Rvdw'))
        return Owner_type, Registration_ID, Car_Transmission, KM_Driven, Fuel_type, Downpayment_amount, Location
    else:
        return "N/A", "N/A","N/A","N/A","N/A","N/A","N/A"

def parse_car_details(soup):
    cars = []
    listings = soup.find_all('a', class_='IIJDn')
    for i in listings:
        try:
            car_info = get_text_or_default(i.find('h3', class_='_11dVb'))
            car_info_parts = car_info.split(' ', 3)
            
            Car_model = car_info_parts[0] if len(car_info_parts) > 0 else "N/A"
            Brand = car_info_parts[1] if len(car_info_parts) > 1 else "N/A"
            Car_Name = car_info_parts[2] if len(car_info_parts) > 2 else "N/A"
            Car_Variant = car_info_parts[3] if len(car_info_parts) > 3 else "N/A"
            Monthly_EMI = get_text_or_default(i.find('span', class_='_2O0yU'))
            Car_Price = get_text_or_default(i.find('strong', class_='_3RL-I'))
            
            # Above details are extracted from main page but once we click on cars it takes us to
            # details page, so extarcting 'href' from 'a' tag.
            
            detail_page_url = i['href']  # Construct the URL for the detailed page
            
#             Car_Transmission = get_text_or_default(i.find('li', class_='_3W_AJ'))
#             KM_Driven = get_text_or_default(i.find('ul', class_='_3J2G-'))
#             Fuel_type = get_text_or_default(i.find('li', class_='_3W_AJ'))
#             Downpayment_amount = get_text_or_default(i.find('label', class_='F6S7B'))
#             Location = get_text_or_default(i.find('p', class_='_3dGMY'))

            # Fetch owner type and registration ID from the detailed page
            Owner_type, Registration_ID, Car_Transmission, KM_Driven, Fuel_type, Downpayment_amount, Location = fetch_detailed_page_details(detail_page_url)

            car = {
                'Car Model': Car_model,
                'Brand': Brand,
                'Car Name': Car_Name,
                'Car Variant': Car_Variant,
                'Car Transmission': Car_Transmission,
                'KM Driven': KM_Driven,
                'Owner Type': Owner_type,
                'Fuel Type': Fuel_type,
                'Registration ID': Registration_ID,
                'Monthly EMI': Monthly_EMI,
                'Car Price': Car_Price,
                'Downpayment Amount': Downpayment_amount,
                'Location': Location
            }
            cars.append(car)
        except AttributeError as e:
            print(f"Error parsing car details: {e}")
            continue
    return cars

def scrape_cars24(location_url, total_rows=40):
    cars_data = []
    page = 1
    last_page_content =None
    while len(cars_data) < total_rows:
        url = f"{location_url}?page={page}"
        print(f"Fetching page {page}...")
        html_content = fetch_page(url)
        if html_content:
            if html_content == last_page_content:
                print('Duplicate page')
                break
            soup = BeautifulSoup(html_content, 'html.parser')
            cars = parse_car_details(soup)
            cars_data.extend(cars)
            print(f"Collected {len(cars_data)} rows so far.")
            page += 1
            time.sleep(2)  # To avoid overwhelming the server
        else:
            break
    return cars_data

# URLs for two metro locations (example URLs, replace with actual ones)
location1_url = 'https://www.cars24.com/buy-used-cars-bangalore/'
location2_url = 'https://www.cars24.com/buy-used-cars-new-delhi/'

# Scrape data for location 1
location1_data = scrape_cars24(location1_url)

# Scrape data for location 2
location2_data = scrape_cars24(location2_url)


Fetching page 1...
Collected 20 rows so far.
Fetching page 2...
Collected 40 rows so far.
Fetching page 1...
Collected 20 rows so far.
Fetching page 2...
Collected 40 rows so far.


In [1]:
# MAIN CODE FOR WEBSCRAPPING

from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from bs4 import BeautifulSoup

# detail url = to get inside car main page( acess 'href' )
def fetch_detailed_page_details(detail_url, driver):
    driver.get(detail_url)
    time.sleep(2)  # Allow time for the page to load
    detailed_html_content = driver.page_source
    detail_soup = BeautifulSoup(detailed_html_content, 'html.parser')
    strong_list = detail_soup.find_all('strong', class_='_3gHeV')     # extracting all strong tags to extract attributes

    Owner_type = strong_list[8].text.strip() if len(strong_list) > 8 else "N/A"   # to avoid None type error
    Registration_ID = strong_list[2].text.strip() if len(strong_list) > 2 else "N/A"
    Car_Transmission = strong_list[6].text.strip() if len(strong_list) > 6 else "N/A"
    KM_Driven = strong_list[7].text.strip() if len(strong_list) > 7 else "N/A"
    Fuel_type = strong_list[9].text.strip() if len(strong_list) > 9 else "N/A"
    Downpayment_amount = detail_soup.find('label', class_='F6S7B').text.strip() if detail_soup.find('label', class_='F6S7B') else "N/A"
    Location = detail_soup.find('li', class_='_1Rvdw').text.strip() if detail_soup.find('li', class_='_1Rvdw') else "N/A"
    
    return Owner_type, Registration_ID, Car_Transmission, KM_Driven, Fuel_type, Downpayment_amount, Location

def scrape_cars24(location_url, total_rows=500):# total rows to extarct
    driver = webdriver.Chrome()                 # establishing chrome webdriver
    driver.get(location_url)
    
    cars_data = []
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while len(cars_data) < total_rows:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(6)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:          # to avoid duplication of content in pages while scrolling
            break
        
        last_height = new_height

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        listings = soup.find_all('a', class_='IIJDn')
        
        for i in listings:
            if len(cars_data) >= total_rows:
                break
            try:
                car_info = i.find('h3', class_='_11dVb').text.strip()
                car_info_parts = car_info.split(' ', 3) #splitting string in h3 tag to get all details
                
                Car_model = car_info_parts[0] if len(car_info_parts) > 0 else "N/A"
                Brand = car_info_parts[1] if len(car_info_parts) > 1 else "N/A"
                Car_Name = car_info_parts[2] if len(car_info_parts) > 2 else "N/A"
                Car_Variant = car_info_parts[3] if len(car_info_parts) > 3 else "N/A"
                Monthly_EMI = i.find('span', class_='_2O0yU').text.strip() if i.find('span', class_='_2O0yU') else "N/A"
                Car_Price = i.find('strong', class_='_3RL-I').text.strip() if i.find('strong', class_='_3RL-I') else "N/A"
                
                detail_page_url = i['href']
                Owner_type, Registration_ID, Car_Transmission, KM_Driven, Fuel_type, Downpayment_amount, Location = fetch_detailed_page_details(detail_page_url, driver)
                
                # dictionary to fill tables
                car = {
                    'Car Model': Car_model,
                    'Brand': Brand,
                    'Car Name': Car_Name,
                    'Car Variant': Car_Variant,
                    'Car Transmission': Car_Transmission,
                    'KM Driven': KM_Driven,
                    'Owner Type': Owner_type,
                    'Fuel Type': Fuel_type,
                    'Registration ID': Registration_ID,
                    'Monthly EMI': Monthly_EMI,
                    'Car Price': Car_Price,
                    'Downpayment Amount': Downpayment_amount,
                    'Location': Location
                }
                cars_data.append(car)
            except AttributeError as e:
                print(f"Error parsing car details: {e}")
                continue
    
    driver.quit()
    return cars_data

# URLs for two metro locations (example URLs, replace with actual ones)
location1_url = 'https://www.cars24.com/buy-used-cars-bangalore/'
location2_url = 'https://www.cars24.com/buy-used-cars-new-delhi/'

# Scrape data for location 1
location1_data = scrape_cars24(location1_url)

# Scrape data for location 2
location2_data = scrape_cars24(location2_url)

In [163]:
# Converting raw data into DataFrame
location1_df=pd.DataFrame(location1_data)
location1_df.head()

,Car Model,Brand,Car Name,Car Variant,Car Transmission,KM Driven,Owner Type,Fuel Type,Registration ID,Monthly EMI,Car Price,Downpayment Amount,Location
0,2017,Hyundai,Creta,SX PLUS AT 1.6 PETROL,Automatic,"98,493 km",1st owner,Petrol,KA03-MZ9224,"EMIs from ₹18,520/month",₹9.73 Lakh,0 down payment,"Parked at: Whitefield Main Road, Bhoruka ..."
1,2018,Renault,Kwid,1.0 MARVEL IRON MAN EDITION AMT,"19,178 km",1st owner,Petrol,N/A,KA03-NC7872,"EMIs from ₹8,192/month",₹4.19 Lakh,0 down payment,"Parked at: Whitefield Main Road, Bhoruka ..."
2,2017,Nissan,Micra,XV CVT,Automatic,"35,474 km",2nd owner,Petrol,KA03-NB5434,"EMIs from ₹10,323/month",₹5.28 Lakh,0 down payment,"Parked at: Hosur Road, Singasandra, Benga..."
3,2017,Hyundai,Eon,ERA PLUS (O),Manual,"33,963 km",2nd owner,Petrol,KA05-MV5787,"EMIs from ₹7,449/month",₹3.81 Lakh,0 down payment,"Parked at: Bellahalli, Bengaluru"
4,2012,Maruti,Swift,VXI,Manual,"64,557 km",1st owner,Petrol,KA41-P4553,"EMIs from ₹15,378/month",₹4.63 Lakh,0 down payment,"Parked at: Bellahalli, Bengaluru"


In [165]:
# DataFrame for location2
location2_df=pd.DataFrame(location2_data)
location2_df.head()

,Car Model,Brand,Car Name,Car Variant,Car Transmission,KM Driven,Owner Type,Fuel Type,Registration ID,Monthly EMI,Car Price,Downpayment Amount,Location
0,2016,Maruti,Alto,800 LXI,Manual,"97,698 km",2nd owner,Petrol,HR72-C5902,"EMIs from ₹4,712/month",₹2.41 Lakh,0 down payment,"Parked at: Chhatarpur, Delhi"
1,2015,Maruti,Swift,VXI,Manual,"102,460 km",1st owner,Petrol,HR26-CS8633,"EMIs from ₹7,560/month",₹3.40 Lakh,0 down payment,"Parked at: Metro Walk, Rohini, New Delhi"
2,2018,Honda,Jazz,1.2L I-VTEC V,"12,425 km",1st owner,Petrol,N/A,DL1C-Z2806,"EMIs from ₹11,183/month",₹5.72 Lakh,0 down payment,"Parked at: Chhatarpur, Delhi"
3,2019,Maruti,Baleno,DELTA PETROL 1.2,Manual,"35,190 km",1st owner,Petrol,DL9C-AR6649,"EMIs from ₹10,889/month",₹5.57 Lakh,0 down payment,"Parked at: Chhatarpur, Delhi"
4,2020,Mahindra,XUV300,W8 (O) 1.5 DIESEL AMT,Automatic,"64,038 km",1st owner,Diesel,DL10-CP1750,"EMIs from ₹17,093/month",₹8.98 Lakh,0 down payment,"Parked at: Chhatarpur, Delhi"


In [167]:
# info for location 1
location1_df.info()

# 13 columns and 340 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Car Model           340 non-null    object
 1   Brand               340 non-null    object
 2   Car Name            340 non-null    object
 3   Car Variant         340 non-null    object
 4   Car Transmission    340 non-null    object
 5   KM Driven           340 non-null    object
 6   Owner Type          340 non-null    object
 7   Fuel Type           340 non-null    object
 8   Registration ID     340 non-null    object
 9   Monthly EMI         340 non-null    object
 10  Car Price           340 non-null    object
 11  Downpayment Amount  340 non-null    object
 12  Location            340 non-null    object
dtypes: object(13)
memory usage: 34.7+ KB


In [169]:
## info for location 2 = NewDelhi
location2_df.info()

# 13 columns and 276 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Car Model           276 non-null    object
 1   Brand               276 non-null    object
 2   Car Name            276 non-null    object
 3   Car Variant         276 non-null    object
 4   Car Transmission    276 non-null    object
 5   KM Driven           276 non-null    object
 6   Owner Type          276 non-null    object
 7   Fuel Type           276 non-null    object
 8   Registration ID     276 non-null    object
 9   Monthly EMI         276 non-null    object
 10  Car Price           276 non-null    object
 11  Downpayment Amount  276 non-null    object
 12  Location            276 non-null    object
dtypes: object(13)
memory usage: 28.2+ KB


In [171]:
# Describe for each location
location1_df.describe()

,Car Model,Brand,Car Name,Car Variant,Car Transmission,KM Driven,Owner Type,Fuel Type,Registration ID,Monthly EMI,Car Price,Downpayment Amount,Location
count,340,340,340,340,340,340,340,340,340,340,340,340,340
unique,14,15,75,229,25,320,6,4,340,312,288,2,6
top,2019,Maruti,Swift,VXI,Manual,1st owner,1st owner,Petrol,KA03-MZ9224,"EMIs from ₹14,917/month",₹4.60 Lakh,0 down payment,"Parked at: Bellahalli, Bengaluru"
freq,39,93,29,15,229,17,200,260,1,3,3,339,158


In [173]:
location2_df.describe()

,Car Model,Brand,Car Name,Car Variant,Car Transmission,KM Driven,Owner Type,Fuel Type,Registration ID,Monthly EMI,Car Price,Downpayment Amount,Location
count,276,276,276,276,276,276,276,276,276,276,276,276,276
unique,15,14,64,177,16,265,5,4,276,252,240,1,2
top,2019,Maruti,Baleno,VXI,Manual,1st owner,1st owner,Petrol,HR72-C5902,"EMIs from ₹12,336/month",₹4.76 Lakh,0 down payment,"Parked at: Metro Walk, Rohini, New Delhi"
freq,53,95,25,8,209,12,211,228,1,3,4,276,206


In [175]:
#
location1_df.duplicated().sum()

0

In [177]:
location2_df.duplicated().sum()

0

In [179]:
# Check for null values
location1_df.isnull().sum().any()

False

In [181]:
location2_df.isnull().sum().any()

False

# DATA CLEANING

In [183]:
# STEP 1: Change Datatypes of Car Price

def convert_price(price_str):
    # Remove the currency symbol and whitespace
    price_str = price_str.replace('₹', '').replace(' ', '')
    price_str = price_str.replace('Lakh', '')
    # Convert the resulting string to a float
    return float(price_str)

# Apply the function to the 'Car Price' column
location1_df['Car Price'] = location1_df['Car Price'].apply(convert_price)

In [185]:
# renaming the column
location1_df.rename(columns={'Car Price': 'Car Price in Lakhs'}, inplace=True)
location1_df.head()

,Car Model,Brand,Car Name,Car Variant,Car Transmission,KM Driven,Owner Type,Fuel Type,Registration ID,Monthly EMI,Car Price in Lakhs,Downpayment Amount,Location
0,2017,Hyundai,Creta,SX PLUS AT 1.6 PETROL,Automatic,"98,493 km",1st owner,Petrol,KA03-MZ9224,"EMIs from ₹18,520/month",9.73,0 down payment,"Parked at: Whitefield Main Road, Bhoruka ..."
1,2018,Renault,Kwid,1.0 MARVEL IRON MAN EDITION AMT,"19,178 km",1st owner,Petrol,N/A,KA03-NC7872,"EMIs from ₹8,192/month",4.19,0 down payment,"Parked at: Whitefield Main Road, Bhoruka ..."
2,2017,Nissan,Micra,XV CVT,Automatic,"35,474 km",2nd owner,Petrol,KA03-NB5434,"EMIs from ₹10,323/month",5.28,0 down payment,"Parked at: Hosur Road, Singasandra, Benga..."
3,2017,Hyundai,Eon,ERA PLUS (O),Manual,"33,963 km",2nd owner,Petrol,KA05-MV5787,"EMIs from ₹7,449/month",3.81,0 down payment,"Parked at: Bellahalli, Bengaluru"
4,2012,Maruti,Swift,VXI,Manual,"64,557 km",1st owner,Petrol,KA41-P4553,"EMIs from ₹15,378/month",4.63,0 down payment,"Parked at: Bellahalli, Bengaluru"


In [187]:
# similarly for location 2

def convert_price(price_str):
    price_str = price_str.replace('₹', '').replace(' ', '')
    price_str = price_str.replace('Lakh', '')
    return float(price_str)
location2_df['Car Price'] = location2_df['Car Price'].apply(convert_price)

In [189]:
location2_df.rename(columns={'Car Price': 'Car Price in Lakhs'}, inplace=True)
location2_df.head()

,Car Model,Brand,Car Name,Car Variant,Car Transmission,KM Driven,Owner Type,Fuel Type,Registration ID,Monthly EMI,Car Price in Lakhs,Downpayment Amount,Location
0,2016,Maruti,Alto,800 LXI,Manual,"97,698 km",2nd owner,Petrol,HR72-C5902,"EMIs from ₹4,712/month",2.41,0 down payment,"Parked at: Chhatarpur, Delhi"
1,2015,Maruti,Swift,VXI,Manual,"102,460 km",1st owner,Petrol,HR26-CS8633,"EMIs from ₹7,560/month",3.40,0 down payment,"Parked at: Metro Walk, Rohini, New Delhi"
2,2018,Honda,Jazz,1.2L I-VTEC V,"12,425 km",1st owner,Petrol,N/A,DL1C-Z2806,"EMIs from ₹11,183/month",5.72,0 down payment,"Parked at: Chhatarpur, Delhi"
3,2019,Maruti,Baleno,DELTA PETROL 1.2,Manual,"35,190 km",1st owner,Petrol,DL9C-AR6649,"EMIs from ₹10,889/month",5.57,0 down payment,"Parked at: Chhatarpur, Delhi"
4,2020,Mahindra,XUV300,W8 (O) 1.5 DIESEL AMT,Automatic,"64,038 km",1st owner,Diesel,DL10-CP1750,"EMIs from ₹17,093/month",8.98,0 down payment,"Parked at: Chhatarpur, Delhi"


In [191]:
# Converting Monthly EMI to float
def convert_emi(emi_str):
    # Remove non-numeric parts and '₹'
    emi_str = emi_str.replace('EMIs from ₹', '').replace('/month', '').replace(',', '')
    # Convert the resulting string to a float
    return float(emi_str)

# Apply the function to the 'Monthly EMI' column
location1_df['Monthly EMI'] = location1_df['Monthly EMI'].apply(convert_emi)

In [193]:
def convert_emi(emi_str):
    # Remove non-numeric parts and '₹'
    emi_str = emi_str.replace('EMIs from ₹', '').replace('/month', '').replace(',', '')
    # Convert the resulting string to a float
    return float(emi_str)

# Apply the function to the 'Monthly EMI' column
location2_df['Monthly EMI'] = location2_df['Monthly EMI'].apply(convert_emi)

In [195]:
location1_df.dtypes

Car Model              object
Brand                  object
Car Name               object
Car Variant            object
Car Transmission       object
KM Driven              object
Owner Type             object
Fuel Type              object
Registration ID        object
Monthly EMI           float64
Car Price in Lakhs    float64
Downpayment Amount     object
Location               object
dtype: object

In [197]:
location2_df.dtypes

Car Model              object
Brand                  object
Car Name               object
Car Variant            object
Car Transmission       object
KM Driven              object
Owner Type             object
Fuel Type              object
Registration ID        object
Monthly EMI           float64
Car Price in Lakhs    float64
Downpayment Amount     object
Location               object
dtype: object

In [201]:
# CONVERTING to csv files
location1_df.to_csv('Cars24_Bengaluru.csv', index=False)
location2_df.to_csv('Cars24_NewDelhi.csv', index=False)

In [203]:
location1_df.head()

,Car Model,Brand,Car Name,Car Variant,Car Transmission,KM Driven,Owner Type,Fuel Type,Registration ID,Monthly EMI,Car Price in Lakhs,Downpayment Amount,Location
0,2017,Hyundai,Creta,SX PLUS AT 1.6 PETROL,Automatic,"98,493 km",1st owner,Petrol,KA03-MZ9224,18520.0,9.73,0 down payment,"Parked at: Whitefield Main Road, Bhoruka ..."
1,2018,Renault,Kwid,1.0 MARVEL IRON MAN EDITION AMT,"19,178 km",1st owner,Petrol,N/A,KA03-NC7872,8192.0,4.19,0 down payment,"Parked at: Whitefield Main Road, Bhoruka ..."
2,2017,Nissan,Micra,XV CVT,Automatic,"35,474 km",2nd owner,Petrol,KA03-NB5434,10323.0,5.28,0 down payment,"Parked at: Hosur Road, Singasandra, Benga..."
3,2017,Hyundai,Eon,ERA PLUS (O),Manual,"33,963 km",2nd owner,Petrol,KA05-MV5787,7449.0,3.81,0 down payment,"Parked at: Bellahalli, Bengaluru"
4,2012,Maruti,Swift,VXI,Manual,"64,557 km",1st owner,Petrol,KA41-P4553,15378.0,4.63,0 down payment,"Parked at: Bellahalli, Bengaluru"
